# Blogpost writing

Apply nested chat conversation arquitecture between several agents to write a blogpost about a theme.

### Setup

In [ ]:
from dotenv import load_dotenv
import os

# Load .env file
load_dotenv(override=True)

# Get environment variables
openai_key = os.getenv("OPENAI_KEY")
openai_endpoint = os.getenv("OPENAI_ENDPOINT")

In [ ]:
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient

az_model_client = AzureOpenAIChatCompletionClient(
    azure_deployment="gpt-4o-mini",
    model="gpt-4o-mini",
    api_version="2024-06-01",
    azure_endpoint=openai_endpoint,
    api_key=openai_key,
)

### Define the task

In [21]:
task = '''
        Write a concise but engaging blogpost about
       AutoGen agents framework. Make sure the blogpost is
       within 100 words.
       '''

### Create a writer agent

In [22]:
from autogen_agentchat.agents import AssistantAgent

writer = AssistantAgent(
    name="Writer",
    system_message="You are a writer. You write engaging and concise " 
        "blogpost (with title) on given topics. You must polish your "
        "writing based on the feedback you receive and give a refined "
        "version. Only return your final work without additional comments.",
    model_client=az_model_client,
)

#### Test writer agent alone

The CancellationToken can be used to cancel the request asynchronously when you call cancellation_token.cancel(), which will cause the await on the on_messages call to raise a CancelledError.

In [23]:
from autogen_agentchat.messages import TextMessage
from autogen_core import CancellationToken

cancellation_token = CancellationToken()
response = await writer.on_messages([TextMessage(content=task, source="user")], cancellation_token)
print(response)

Response(chat_message=TextMessage(source='Writer', models_usage=RequestUsage(prompt_tokens=89, completion_tokens=120), content='**Unlocking Creativity with AutoGen Agents**\n\nThe AutoGen agents framework revolutionizes the way we interact with AI by enabling the creation of autonomous agents that can generate content and solve complex tasks. These agents autonomously collaborate, utilizing their unique strengths to produce innovative solutions across various domains. With features like self-organization and adaptability, AutoGen empowers developers to build dynamic systems that respond effectively to user needs. Whether in customer service, content creation, or data analysis, the potential applications are limitless. Dive into the world of AutoGen and discover how it can elevate your projects and stimulate creativity like never before!', type='TextMessage'), inner_messages=[])


In [24]:
from IPython.display import Markdown
Markdown(response.chat_message.content)

**Unlocking Creativity with AutoGen Agents**

The AutoGen agents framework revolutionizes the way we interact with AI by enabling the creation of autonomous agents that can generate content and solve complex tasks. These agents autonomously collaborate, utilizing their unique strengths to produce innovative solutions across various domains. With features like self-organization and adaptability, AutoGen empowers developers to build dynamic systems that respond effectively to user needs. Whether in customer service, content creation, or data analysis, the potential applications are limitless. Dive into the world of AutoGen and discover how it can elevate your projects and stimulate creativity like never before!

### Adding reflection - two-agent chat

Create a critic agent to reflect on the work (feedback) of the writer agent.

TO-DO: Mirar llamada para quedarme sólo con el último mensaje de la conversación, es decir, el blog final del agente escritor.

In [25]:
critic = AssistantAgent(
    name="Critic",
    system_message="You are a critic. You review the work of "
                "the writer and provide constructive "
                "feedback to help improve the quality of the content.",
    model_client=az_model_client,
)

In [26]:
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import TextMentionTermination, MaxMessageTermination
from autogen_agentchat.ui import Console

# The termination condition is a combination of text termination and max message termination, either of which will cause the chat to terminate.
termination = MaxMessageTermination(4)

# The group chat will alternate between the assistant and the code executor.
group_chat = RoundRobinGroupChat([writer, critic], termination_condition=termination)

# `run_stream` returns an async generator to stream the intermediate messages.
stream = group_chat.run_stream(task=task)
# `Console` is a simple UI to display the stream.
await Console(stream)

---------- user ----------

        Write a concise but engaging blogpost about
       AutoGen agents framework. Make sure the blogpost is
       within 100 words.
       
---------- Writer ----------
**Harnessing the Power of AutoGen Agents**

The AutoGen agents framework is transforming the landscape of artificial intelligence by enabling the development of autonomous agents capable of generating intelligent content and executing complex tasks. These agents collaborate efficiently, leveraging their individual strengths to innovate solutions across diverse applications. With a focus on self-organization and adaptability, AutoGen allows developers to create responsive systems that cater to user requirements seamlessly. From enhancing customer engagement to streamlining data analysis, the possibilities are endless. Explore AutoGen today and unlock new heights of creativity and efficiency in your projects!
---------- Critic ----------
This blog post is engaging and informative, effective

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='\n        Write a concise but engaging blogpost about\n       AutoGen agents framework. Make sure the blogpost is\n       within 100 words.\n       ', type='TextMessage'), TextMessage(source='Writer', models_usage=RequestUsage(prompt_tokens=251, completion_tokens=111), content='**Harnessing the Power of AutoGen Agents**\n\nThe AutoGen agents framework is transforming the landscape of artificial intelligence by enabling the development of autonomous agents capable of generating intelligent content and executing complex tasks. These agents collaborate efficiently, leveraging their individual strengths to innovate solutions across diverse applications. With a focus on self-organization and adaptability, AutoGen allows developers to create responsive systems that cater to user requirements seamlessly. From enhancing customer engagement to streamlining data analysis, the possibilities are endless. Explore AutoGen to

### Adding reflection - nested chat

Create a nested chat "inside" the critic agent. There will be a whole team of more specific agents inside this critic agent, to enhance results and take specific aspect into account.

SocietyOfMindAgent: An agent that uses an inner team of agents to generate responses.

In [27]:
SEO_reviewer = AssistantAgent(
    name="SEO_reviewer",
    system_message="You are an SEO reviewer, known for "
        "your ability to optimize content for search engines, "
        "ensuring that it ranks well and attracts organic traffic. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
    model_client=az_model_client,
)

In [28]:
legal_reviewer = AssistantAgent(
    name="Legal_Reviewer",
    system_message="You are a legal reviewer, known for "
        "your ability to ensure that content is legally compliant "
        "and free from any potential legal issues. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
    model_client=az_model_client,
)

In [29]:
ethics_reviewer = AssistantAgent(
    name="Ethics_Reviewer",
    system_message="You are an ethics reviewer, known for "
        "your ability to ensure that content is ethically sound "
        "and free from any potential ethical issues. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
    model_client=az_model_client,
)

In [31]:
from autogen_agentchat.agents import SocietyOfMindAgent
inner_termination = MaxMessageTermination(5)
inner_team = RoundRobinGroupChat([SEO_reviewer, legal_reviewer, ethics_reviewer], termination_condition=inner_termination)
critic = SocietyOfMindAgent("critic", 
                                team=inner_team, 
                                model_client=az_model_client, 
                                description = "An agent that uses an inner team of agents to review the work of "
                                            "the writer and provide constructive "
                                            "feedback to help improve the quality of the content.")

In [32]:
# The termination condition is a combination of text termination and max message termination, either of which will cause the chat to terminate.
termination = TextMentionTermination("TERMINATE") | MaxMessageTermination(7)

# The group chat will alternate between the assistant and the code executor.
group_chat = RoundRobinGroupChat([writer, critic], termination_condition=termination)

# `run_stream` returns an async generator to stream the intermediate messages.
stream = group_chat.run_stream(task=task)
# `Console` is a simple UI to display the stream.
await Console(stream)

---------- user ----------

        Write a concise but engaging blogpost about
       AutoGen agents framework. Make sure the blogpost is
       within 100 words.
       
---------- Writer ----------
**Discover the Future with AutoGen Agents Framework**

Are you curious about the next wave of AI innovation? The AutoGen agents framework is pioneering a new era, allowing autonomous agents to generate content and tackle intricate tasks independently. With their ability to collaborate and adapt, these agents optimize efficiency across diverse industries, from customer service to creative content generation. For example, they can handle complex inquiries in real-time, enhancing user satisfaction. The potential is limitless! Explore AutoGen and see how its groundbreaking capabilities are transforming workflows and unlocking creative solutions that redefine what's possible in technology today!
---------- SEO_reviewer ----------
As an SEO reviewer, here are my suggestions for optimizing your 

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='\n        Write a concise but engaging blogpost about\n       AutoGen agents framework. Make sure the blogpost is\n       within 100 words.\n       ', type='TextMessage'), TextMessage(source='Writer', models_usage=RequestUsage(prompt_tokens=722, completion_tokens=113), content="**Discover the Future with AutoGen Agents Framework**\n\nAre you curious about the next wave of AI innovation? The AutoGen agents framework is pioneering a new era, allowing autonomous agents to generate content and tackle intricate tasks independently. With their ability to collaborate and adapt, these agents optimize efficiency across diverse industries, from customer service to creative content generation. For example, they can handle complex inquiries in real-time, enhancing user satisfaction. The potential is limitless! Explore AutoGen and see how its groundbreaking capabilities are transforming workflows and unlocking creative solu